# Load main badcal database from old main website as votable and sync it with TAP badcal tables using catalogAPI (prod and preprod)

In [1]:
from a2p2.jmmc import Catalog
from astropy.io.votable import parse

badcal_vot = parse("http://apps.jmmc.fr/badcal-dsa/SubmitCone?DSACATTAB=badcal.valid_stars&RA=0.0&DEC=0.0&SR=360.0")
org_table  = badcal_vot.get_first_table().to_table().to_pandas()

for prod in (False, True):
    print(f"# Cheking instance prod={prod} of TAP badcal")

    badcal_cat=Catalog("badcal", prod=prod)
    dest_table=badcal_cat.getDataFrame()
    
    dest_ids = dest_table[dest_table.columns[0]]
    org_ids = org_table[org_table.columns[0]]
    missing_records = org_table[~org_ids.isin(dest_ids)]
    
    # Prepate update dictionnary
    org_cname2dest_cname = dict(zip(org_table.columns,dest_table.columns))
    missing_records.rename(columns=org_cname2dest_cname, inplace=True)
    update_dict = missing_records.to_dict('records')
    if update_dict:
        print(f"Updating Badcal with {update_dict}")
        badcal_cat.addRows(update_dict)
    else:
        print(f"  Badcal(prod={prod}) is synchronized")

INFO - a2p2.jmmc.catalogs  - 2025-07-07 10:32:05,786 - catalogs.py:58 - Create catalog wrapper to access 'badcal' (pre-prod API at https://oidb-beta.jmmc.fr/restxq/catalogs)
INFO - a2p2.jmmc.catalogs  - 2025-07-07 10:32:05,787 - catalogs.py:146 - Querying remote catalog : SELECT TOP 10000 * FROM badcal
/tmp/ipykernel_56389/1907440205.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_records.rename(columns=org_cname2dest_cname, inplace=True)
INFO - a2p2.jmmc.catalogs  - 2025-07-07 10:32:05,825 - catalogs.py:58 - Create catalog wrapper to access 'badcal' (production API at https://oidb.jmmc.fr/restxq/catalogs)
INFO - a2p2.jmmc.catalogs  - 2025-07-07 10:32:05,826 - catalogs.py:146 - Querying remote catalog : SELECT TOP 10000 * FROM badcal


# Cheking instance prod=False of TAP badcal
  Badcal(prod=False) is synchronized
# Cheking instance prod=True of TAP badcal
  Badcal(prod=True) is synchronized


/tmp/ipykernel_56389/1907440205.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_records.rename(columns=org_cname2dest_cname, inplace=True)
